In [345]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import os,json
import csv
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.util import bigrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [305]:
from google.colab import files
uploaded = files.upload()

Saving web-1.json to web-1 (1).json
Saving web-3.json to web-3 (1).json
Saving web-4.json to web-4 (1).json
Saving web-1005.json to web-1005 (1).json
Saving web-2012.json to web-2012 (1).json
Saving web-10034.json to web-10034 (1).json
Saving web-10047.json to web-10047 (1).json
Saving web-10089.json to web-10089 (1).json
Saving web-10133.json to web-10133 (1).json
Saving web-10165.json to web-10165 (1).json


In [346]:
json_list = ["web-1.json", "web-3.json", "web-4.json", "web-1005.json", "web-2012.json", "web-10034.json", "web-10047.json", "web-10089.json", "web-10133.json", "web-10165.json"]

data_list=[]
for i in json_list:
  data_list.append(uploaded[i].decode("utf-8"))
data_list

['{"text": ["2021 Coachmen Sportscoach 402TS Two Full Bath, Bunk Beds,Theater Seating, King, W/D", "going back", "Back", "California Privacy Rights", "California Consumer Privacy Act Notice for California Consumers", "800-335-6054", "817-790-7771", "Sitemap", "This page took too long to load. Please try refreshing or ", "going back", " to the previous page.", "All material copyright \\u00a9 Motor Home Specialist ( MHSRV.com ). All rights are reserved. No part of any material on this web site may be reproduced, distributed, or transmitted in any form or by any means without the prior written permission of Motor Home Specialist. *Information deemed reliable, but not guaranteed. Features & options subject to change without notice. Weights & measurements are estimates only. Verify before purchase.", "\\n        ", "*DISCLAIMER:", "\\n        *#1 in the world or #1 in Texas references are per the official Stats Surveys Inc. for American built Motorhomes sold at single location. *MINIMUM 25%

In [347]:
df = pd.DataFrame(zip(data_list, data_list),
               columns =['Raw_Text', 'Processed_Text'])
df.head(10)

,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...","{""text"": [""2021 Coachmen Sportscoach 402TS Two..."
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...","{""text"": [""Jemarl Baker Jr. to transfer from A..."
2,"{""text"": [""Publications | The Protect Heritage...","{""text"": [""Publications | The Protect Heritage..."
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...","{""text"": ["" Arts vs. Athletics: Two Great Ways..."
4,"{\n ""title"": [\n ""Purposeful Investi...","{\n ""title"": [\n ""Purposeful Investi..."
5,"{""text"": [""Our Online Privacy Policy"", ""Skip t...","{""text"": [""Our Online Privacy Policy"", ""Skip t..."
6,"{""text"": [""Python: parsing PDF text and tables...","{""text"": [""Python: parsing PDF text and tables..."
7,"{""text"": [""Bubble Gum Pop Hat | AllFreeCrochet...","{""text"": [""Bubble Gum Pop Hat | AllFreeCrochet..."
8,"{""text"": [""Transitional Care Gets a Room of It...","{""text"": [""Transitional Care Gets a Room of It..."
9,"{""text"": [""Decorative Objects/Accessories - Pa...","{""text"": [""Decorative Objects/Accessories - Pa..."


# **NLP-Pipeline**

**Data Cleansing**

In [348]:
for i in range(10):
  df['Processed_Text'].values[i] = re.sub(r'^https?:\/\/.*[\r\n]*', '', df['Processed_Text'].values[i], flags=re.MULTILINE)
  df['Processed_Text'].values[i] = re.sub(r"\W", " ", df['Processed_Text'].values[i])
  df['Processed_Text'].values[i] = re.sub(r" n ", " ", df['Processed_Text'].values[i])
  df['Processed_Text'].values[i] = re.sub(r" r ", " ", df['Processed_Text'].values[i])
  df['Processed_Text'].values[i] = " ".join(df['Processed_Text'].values[i].split())
  df['Processed_Text'].values[i] = df['Processed_Text'].values[i].split(' ', 1)[1]
  for j in range(5):
    df['Processed_Text'].values[j] = re.sub(r" t ", " ", df['Processed_Text'].values[j])
  df=df.replace('\*','',regex=True)
df["Processed_Text"][0]

'2021 Coachmen Sportscoach 402TS Two Full Bath Bunk Beds Theater Seating King W D going back Back California Privacy Rights California Consumer Privacy Act Notice for California Consumers 800 335 6054 817 790 7771 Sitemap This page took too long to load Please try refreshing or going back to the previous page All material copyright u00a9 Motor Home Specialist MHSRV com All rights are reserved No part of any material on this web site may be reproduced distributed or transmitted in any form or by any means without the prior written permission of Motor Home Specialist Information deemed reliable but not guaranteed Features options subject to change without notice Weights measurements are estimates only Verify before purchase DISCLAIMER 1 in the world or 1 in Texas references are per the official Stats Surveys Inc for American built Motorhomes sold at single location MINIMUM 25 OFF MSRP DISCOUNT DOES NOT APPLY TO CLASS B RVS FORESTER DYNAMAX OR TWILIGHT PRODUCTS The discount shown on a uni

In [349]:
for i in range(10):
  # 'inplace =True' is used to change the source dataframe
  df["Processed_Text"].replace('\d+', '', regex=True, inplace = True)
df

,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...",Coachmen Sportscoach TS Two Full Bath Bunk Be...
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...",Jemarl Baker Jr to transfer from Arizona Wildc...
2,"{""text"": [""Publications | The Protect Heritage...",Publications The Protect Heritage Corp Home Ab...
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...",Arts vs Athletics Two Great Ways You Can Prepa...
4,"{\n ""title"": [\n ""Purposeful Investi...",Purposeful Investing Legacy Bridge Private Fam...
5,"{""text"": [""Our Online Privacy Policy"", ""Skip t...",Our Online Privacy Policy Skip to main content...
6,"{""text"": [""Python: parsing PDF text and tables...",Python parsing PDF text and tables usage and c...
7,"{""text"": [""Bubble Gum Pop Hat | AllFreeCrochet...",Bubble Gum Pop Hat AllFreeCrochet com close Ad...
8,"{""text"": [""Transitional Care Gets a Room of It...",Transitional Care Gets a Room of Its Own Patie...
9,"{""text"": [""Decorative Objects/Accessories - Pa...",Decorative Objects Accessories Page of Engli...


**Stop Words Removal**

In [350]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stop = stopwords.words('english')
stop.append('n')
stop.append('r')
lem = WordNetLemmatizer()
stem = PorterStemmer()
for i in range(10):
   x = df['Processed_Text'].values[i]
   token = nltk.word_tokenize(x)
   df['Processed_Text'].values[i] = token
   c = [t for t in token if not t in stop]
   df['Processed_Text'].values[i] = c
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...","[Coachmen, Sportscoach, TS, Two, Full, Bath, B..."
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...","[Jemarl, Baker, Jr, transfer, Arizona, Wildcat..."
2,"{""text"": [""Publications | The Protect Heritage...","[Publications, The, Protect, Heritage, Corp, H..."
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...","[Arts, vs, Athletics, Two, Great, Ways, You, C..."
4,"{\n ""title"": [\n ""Purposeful Investi...","[Purposeful, Investing, Legacy, Bridge, Privat..."


**Tokenization**

In [343]:
def tokenizing(Processed_Text):
  split = re.split('\W+', Processed_Text)
  return split

In [ ]:
df_data['Text']=df_data['Text'].apply(lambda x: tokenizing(x.lower()))

In [331]:
df["Processed_Text"][0]

[' ',
 'C',
 'c',
 'h',
 'e',
 'n',
 ' ',
 'S',
 'p',
 'r',
 'c',
 'c',
 'h',
 ' ',
 'T',
 'S',
 ' ',
 'T',
 'w',
 ' ',
 'F',
 'u',
 'l',
 'l',
 ' ',
 'B',
 'h',
 ' ',
 'B',
 'u',
 'n',
 'k',
 ' ',
 'B',
 'e',
 ' ',
 'T',
 'h',
 'e',
 'e',
 'r',
 ' ',
 'S',
 'e',
 'n',
 'g',
 ' ',
 'K',
 'n',
 'g',
 ' ',
 'W',
 ' ',
 'D',
 ' ',
 'g',
 'n',
 'g',
 ' ',
 'b',
 'c',
 'k',
 ' ',
 'B',
 'c',
 'k',
 ' ',
 'C',
 'l',
 'f',
 'r',
 'n',
 ' ',
 'P',
 'r',
 'v',
 'c',
 ' ',
 'R',
 'g',
 'h',
 ' ',
 'C',
 'l',
 'f',
 'r',
 'n',
 ' ',
 'C',
 'n',
 'u',
 'e',
 'r',
 ' ',
 'P',
 'r',
 'v',
 'c',
 ' ',
 'A',
 'c',
 ' ',
 'N',
 'c',
 'e',
 ' ',
 'f',
 'r',
 ' ',
 'C',
 'l',
 'f',
 'r',
 'n',
 ' ',
 'C',
 'n',
 'u',
 'e',
 'r',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 'S',
 'e',
 'p',
 ' ',
 'T',
 'h',
 ' ',
 'p',
 'g',
 'e',
 ' ',
 'k',
 ' ',
 ' ',
 'l',
 'n',
 'g',
 ' ',
 ' ',
 'l',
 ' ',
 'P',
 'l',
 'e',
 'e',
 ' ',
 'r',
 ' ',
 'r',
 'e',
 'f',
 'r',
 'e',
 'h',
 'n',
 'g',
 ' ',
 'r',
 ' ',
 'g'

**Stemming**

In [354]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

for x in range(10):
  z = []
  y = df['Processed_Text'].values[x]
  for j in y:
    z.append(stem.stem(j))
  df['Processed_Text'].values[x] = z
df.head()

,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...","[coachmen, sportscoach, TS, two, full, bath, b..."
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...","[jemarl, baker, Jr, transfer, arizona, wildcat..."
2,"{""text"": [""Publications | The Protect Heritage...","[public, the, protect, heritag, corp, home, ab..."
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...","[art, v, athlet, two, great, way, you, can, pr..."
4,"{\n ""title"": [\n ""Purposeful Investi...","[purpos, invest, legaci, bridg, privat, famili..."


**Lemmatization**

In [353]:
nltk.download('wordnet')
for x in range(10):
  z = []
  y = df['Processed_Text'].values[x]
  for j in y:
    z.append(lem.lemmatize(j))
  df['Processed_Text'].values[x] = z
df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...","[Coachmen, Sportscoach, TS, Two, Full, Bath, B..."
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...","[Jemarl, Baker, Jr, transfer, Arizona, Wildcat..."
2,"{""text"": [""Publications | The Protect Heritage...","[Publications, The, Protect, Heritage, Corp, H..."
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...","[Arts, v, Athletics, Two, Great, Ways, You, Ca..."
4,"{\n ""title"": [\n ""Purposeful Investi...","[Purposeful, Investing, Legacy, Bridge, Privat..."


**Bigram Formation**

In [ ]:
from nltk.util import bigrams
for i in range(10):
  str = df["Processed_Text"].values[i]
  bigram_list = nltk.bigrams(str)
  bigram_list = [" ".join(bigram) for bigram in bigram_list]
  df["Processed_Text"].values[i] = bigram_list

In [ ]:
df["Processed_Text"][1]

['J e',
 'e r',
 'r l',
 'l  ',
 '  B',
 'B k',
 'k e',
 'e r',
 'r  ',
 '  J',
 'J r',
 'r  ',
 '   ',
 '  r',
 'r n',
 'n f',
 'f e',
 'e r',
 'r  ',
 '  f',
 'f r',
 'r  ',
 '  A',
 'A r',
 'r z',
 'z n',
 'n  ',
 '  W',
 'W l',
 'l c',
 'c  ',
 '  p',
 'p e',
 'e r',
 'r  ',
 '  r',
 'r e',
 'e p',
 'p r',
 'r  ',
 '  A',
 'A r',
 'r z',
 'z n',
 'n  ',
 '  D',
 'D e',
 'e e',
 'e r',
 'r  ',
 '  S',
 'S w',
 'w r',
 'r  ',
 '  S',
 'S k',
 'k p',
 'p  ',
 '   ',
 '  n',
 'n  ',
 '  c',
 'c n',
 'n e',
 'e n',
 'n  ',
 '  L',
 'L g',
 'g  ',
 '  I',
 'I n',
 'n  ',
 '  S',
 'S g',
 'g n',
 'n  ',
 '  U',
 'U p',
 'p  ',
 '  S',
 'S e',
 'e c',
 'c n',
 'n  ',
 '  A',
 'A Z',
 'Z D',
 'D S',
 'S  ',
 '  S',
 'S f',
 'f f',
 'f  ',
 '  F',
 'F n',
 'n p',
 'p  ',
 '  W',
 'W l',
 'l c',
 'c  ',
 '  F',
 'F n',
 'n h',
 'h  ',
 '  S',
 'S h',
 'h p',
 'p  ',
 '  O',
 'O  ',
 '  F',
 'F n',
 'n  ',
 '  A',
 'A b',
 'b u',
 'u  ',
 '  C',
 'C u',
 'u n',
 'n  ',
 '  G',
 'G u',
 'u e',


In [ ]:
df.head(10)

,Raw_Text,Processed_Text
0,"{""text"": [""2021 Coachmen Sportscoach 402TS Two...","[2 0, 0 2, 2 1, 1 , C, C c, c h, h e, e n, ..."
1,"{""text"": [""Jemarl Baker Jr. to transfer from A...","[J e, e r, r l, l , B, B k, k e, e r, r , ..."
2,"{""text"": [""Publications | The Protect Heritage...","[P u, u b, b l, l c, c n, n , T, T h, h e, ..."
3,"{""text"": ["" Arts vs. Athletics: Two Great Ways...","[A r, r , v, v , A, A h, h l, l e, e c, ..."
4,"{\n ""title"": [\n ""Purposeful Investi...","[P u, u r, r p, p e, e f, f u, u l, l , I, ..."
5,"{""text"": [""Our Online Privacy Policy"", ""Skip t...","[O u, u r, r , O, O n, n l, l n, n e, e , ..."
6,"{""text"": [""Python: parsing PDF text and tables...","[P h, h n, n , p, p r, r n, n g, g , P, ..."
7,"{""text"": [""Bubble Gum Pop Hat | AllFreeCrochet...","[B u, u b, b b, b l, l e, e , G, G u, u , ..."
8,"{""text"": [""Transitional Care Gets a Room of It...","[T r, r n, n n, n l, l , C, C r, r e, e , ..."
9,"{""text"": [""Decorative Objects/Accessories - Pa...","[D e, e c, c r, r v, v e, e , O, O b, b j, ..."


# **Embedding**

**Bag of words**

In [351]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english', lowercase=True)
for i in range(10):
    y = df['Processed_Text'].values[i]
    word_count = cv.fit_transform(y)
    df_data = pd.DataFrame(word_count.toarray(), columns = cv.get_feature_names())
    print(df_data)

     _cdlwmybcpnsuadupwbxgbwytpsecoejgzhe  ...  zgztrrodzwhhikozlpndjjppgvvozzpz
0                                       0  ...                                 0
1                                       0  ...                                 0
2                                       0  ...                                 0
3                                       0  ...                                 0
4                                       0  ...                                 0
..                                    ...  ...                               ...
483                                     0  ...                                 0
484                                     0  ...                                 0
485                                     0  ...                                 0
486                                     0  ...                                 0
487                                     1  ...                                 0

[488 rows x 241 columns]
  

**Term Frequency Inverse Document Frequency(TF-IDF)**

In [352]:
from sklearn.feature_extraction.text import TfidfVectorizer
Tf_IDF = TfidfVectorizer()
for i in range(10):
  print("\nDOCCUMENT: ", i)
  temp = df["Processed_Text"].values[i]
  TF_IDF_val = Tf_IDF.fit_transform(temp)
  print("TF_IDF Values: \n")
  print(TF_IDF_val)


DOCCUMENT:  0
TF_IDF Values: 

  (0, 30)	1.0
  (1, 207)	1.0
  (2, 231)	1.0
  (3, 237)	1.0
  (4, 75)	1.0
  (5, 14)	1.0
  (6, 19)	1.0
  (7, 16)	1.0
  (8, 223)	1.0
  (9, 190)	1.0
  (10, 96)	1.0
  (13, 77)	1.0
  (14, 13)	1.0
  (15, 13)	1.0
  (16, 21)	1.0
  (17, 158)	1.0
  (18, 181)	1.0
  (19, 21)	1.0
  (20, 33)	1.0
  (21, 158)	1.0
  (22, 2)	1.0
  (23, 130)	1.0
  (24, 21)	1.0
  (25, 34)	1.0
  (26, 199)	1.0
  :	:
  (462, 14)	1.0
  (463, 19)	1.0
  (464, 16)	1.0
  (465, 223)	1.0
  (466, 190)	1.0
  (467, 96)	1.0
  (469, 126)	1.0
  (470, 46)	1.0
  (471, 165)	1.0
  (472, 238)	1.0
  (473, 86)	1.0
  (474, 118)	1.0
  (475, 213)	1.0
  (476, 84)	1.0
  (477, 118)	1.0
  (478, 31)	1.0
  (479, 86)	1.0
  (480, 103)	1.0
  (481, 103)	1.0
  (482, 118)	1.0
  (483, 107)	1.0
  (484, 220)	1.0
  (485, 152)	1.0
  (486, 133)	1.0
  (487, 0)	1.0

DOCCUMENT:  1
TF_IDF Values: 

  (0, 120)	1.0
  (1, 22)	1.0
  (2, 124)	1.0
  (3, 257)	1.0
  (4, 11)	1.0
  (5, 282)	1.0
  (6, 169)	1.0
  (7, 195)	1.0
  (8, 11)	1.0
  (9, 58)	